# Visualize a GeoPandas dataframe in Colab

get a gpd dataframe

In [ ]:
import duckdb
from shapely import wkb
import geopandas as gpd

duckdb.sql("install spatial; load spatial")

LIMIT = 1000

pdf = duckdb.sql(
    f"""
select * replace (st_aswkb(geometry) as geometry)
from
  read_parquet(
    's3://overturemaps-us-west-2/release/2025-05-21.0/theme=buildings/type=building/*',
    filename=true,
    hive_partitioning=1)
WHERE names.primary IS NOT NULL
  AND bbox.xmin BETWEEN -84.36 AND -82.42
  AND bbox.ymin BETWEEN 41.71 AND 43.33
limit {LIMIT}"""
).df()

pdf["geometry"] = pdf["geometry"].apply(lambda x: wkb.loads(bytes(x)))
gdf = gpd.GeoDataFrame(pdf, geometry="geometry").set_crs("OGC:CRS84")

gdf.shape[0]

visualize

# Folium

In [ ]:
!pip install mapclassify --quiet

In [ ]:
gdf.explore()

# Lonboard

In [ ]:
!pip install lonboard --quiet


In [ ]:
from lonboard import viz

viz(gdf)

# kepler

In [ ]:
!pip install keplergl==0.3.2 --quiet


In [ ]:
from google.colab import output

output.enable_custom_widget_manager()

In [ ]:
from keplergl import KeplerGl

m = KeplerGl(height=1000)
m.add_data(data=gdf[["geometry"]])
m

# pydeck

In [ ]:
!pip install pydeck --quiet


In [ ]:
import pydeck as pdk

geojson_layer = pdk.Layer(
    "GeoJsonLayer",
    gdf[["geometry"]],
    pickable=True,
    stroked=True,
    autoHighlight=True,
    get_fill_color=[200, 200, 255, 200],
    extruded=True,
    elevation_scale=0.005,
)

pdk.Deck(layers=[geojson_layer])